<a href="https://colab.research.google.com/github/Sristi222/FakeCurrencyDetector/blob/main/Fakecurrency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/Indian Currency Dataset'


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import shutil
from PIL import Image
import numpy as np

In [10]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [14]:
train = os.path.join(folder_path, 'train')
validation = os.path.join(folder_path, 'validation')
test = os.path.join(folder_path, 'test')


In [15]:
print("Train folders:", os.listdir(train))
print("Validation folders:", os.listdir(validation))
print("Test folders:", os.listdir(test))

Train folders: ['fake', 'real']
Validation folders: ['real', 'fake']
Test folders: ['fake', 'real']


In [16]:
batch_size = 32
target_size = (224, 224)

train_generator = datagen.flow_from_directory(
    train,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed=42
)

validation_generator = datagen.flow_from_directory(
    validation,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,
    seed=42
)

test_generator = datagen.flow_from_directory(
    test,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,
    seed=42
)

Found 150 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Found 107 images belonging to 2 classes.


In [17]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False)
base_model.trainable = False  # Freeze the base model


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [18]:
flatten_layer = tf.keras.layers.Flatten()(base_model.output)
dropout_layer = tf.keras.layers.Dropout(0.5)(flatten_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dropout_layer)

In [19]:
model = tf.keras.models.Model(base_model.input, output_layer)

In [20]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
num_epochs = 10
model.fit(train_generator, epochs=num_epochs, validation_data=validation_generator)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 81s 10s/step - accuracy: 0.5267 - loss: 2.1177 - val_accuracy: 0.7500 - val_loss: 1.0430
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.7225 - loss: 1.3537 - val_accuracy: 0.7333 - val_loss: 0.6260
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 4s/step - accuracy: 0.8063 - loss: 0.8528 - val_accuracy: 0.8167 - val_loss: 0.7813
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 35s 3s/step - accuracy: 0.7432 - loss: 1.2488 - val_accuracy: 0.7333 - val_loss: 1.0738
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 25s 4s/step - accuracy: 0.8025 - loss: 0.8636 - val_accuracy: 0.7667 - val_loss: 0.6814
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 43s 4s/step - accuracy: 0.9168 - loss: 0.5487 - val_accuracy: 0.9000 - val_loss: 0.3575
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.8591 - loss: 0.6145 - val_accuracy: 0.8833 - val_loss: 0.3140
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.8977 - loss: 0.3628 - val_accuracy: 0.9167 - val_loss: 0.3973
Epoch 9/10
5/5 ━━━

In [22]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 4s/step - accuracy: 0.8358 - loss: 0.5318
Test Loss: 0.37314584851264954, Test Accuracy: 0.8785046935081482


In [26]:


def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path)

    # Convert RGBA to RGB if needed
    if img.mode == 'RGBA':
        img = img.convert('RGB')

    # Resize the image to the target size
    img = img.resize(target_size)

    # Normalize pixel values to [0, 1]
    img_array = np.array(img) / 255.0

    # Expand dimensions to match the model's input shape (1, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array
def predict_currency(image_path, model):
    preprocessed_image = load_and_preprocess_image(image_path)
    prediction = model.predict(preprocessed_image)
    if prediction[0][0] >= 0.5:
        return "Real Nepali Currency"
    else:
        return "Fake Nepali Currency"

In [28]:
example_image_real = '/content/drive/MyDrive/Colab Notebooks/Indian Currency Dataset/validation/real/NEPALI REAL NOTE.png'
result = predict_currency(example_image_real, model)
print(f"The predicted result is: {result}")

example_image_fake = '/content/drive/MyDrive/Colab Notebooks/Indian Currency Dataset/validation/fake/1 (1) - Copy.jpg'
result = predict_currency(example_image_fake, model)
print(f"The predicted result is: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
The predicted result is: Real Nepali Currency
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
The predicted result is: Fake Nepali Currency
